Install Pytorch package (need to specify the version to get correctly installed. Possible error will lead died kernal.
)

In [1]:
# pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0

The RNN model with PyTorch will be implemented on the Metadata only to farmiliarize myself with the PyTorch library and get starts with RNN. First of all, we need to treat each patient as an observation and get every 

In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score

A better explanation about the RNN example https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

Read in data set

In [3]:
mydata = pd.read_csv('/Users/mli171/Desktop/JHU/3Summer2022_JHU/DREAM/metadata_imputed.csv', delimiter=',')
mydata = pd.DataFrame(mydata)
mydata

,participant_id,project,specimen,was_term,delivery_wk,collect_wk,race,age,NIH.Racial.Category,NIH.Ethnicity.Category,was_preterm,was_early_preterm,collect_tri,age_imp,race_imp,age_imp_cat
0,A00001,A,A00001-05,True,38,33,American Indian,Unknown,American Indian or Alaska Native,Unknown,False,False,3,27,American Indian or Alaska Native,from_18_to_28
1,A00002,A,A00002-01,True,40,38,White,Unknown,White,Unknown,False,False,3,24,White,from_18_to_28
2,A00003,A,A00003-02,True,40,30,Asian-Japanese,Unknown,Asian,Unknown,False,False,3,32,Asian,from_29_to_38
3,A00004,A,A00004-08,True,40,27,White,Unknown,White,Unknown,False,False,3,25,White,from_18_to_28
4,A00004,A,A00004-12,True,40,29,White,Unknown,White,Unknown,False,False,3,25,White,from_18_to_28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,J00111,J,J00111-01,True,40,17,Caucasian,27,White,Unknown,False,False,2,27,White,from_18_to_28
3574,J00112,J,J00112-01,True,39,19,Caucasian,27,White,Unknown,False,False,2,27,White,from_18_to_28
3575,J00113,J,J00113-01,True,41,16,Caucasian,32,White,Unknown,False,False,2,32,White,from_29_to_38
3576,J00115,J,J00115-01,True,42,18,Caucasian,35,White,Unknown,False,False,2,35,White,from_29_to_38


In [4]:
mydata = mydata[["participant_id", "project", "specimen", "collect_wk", 
                 "was_preterm", "was_early_preterm", "race_imp", "age_imp_cat" ]]
mydata["project"] = mydata["project"].astype('category')
mydata["race_imp"] = mydata["race_imp"].astype('category')
mydata["age_imp_cat"] = mydata["age_imp_cat"].astype('category')

mydata["project"] = mydata["project"].cat.codes
mydata["race_imp"] = mydata["race_imp"].cat.codes
mydata["age_imp_cat"] = mydata["age_imp_cat"].cat.codes
mydata['was_preterm'] = mydata['was_preterm'].astype('int8')
mydata['was_early_preterm'] = mydata['was_early_preterm'].astype('int8')
mydata.dtypes

/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_13597/1563144068.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydata["project"] = mydata["project"].astype('category')
/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_13597/1563144068.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydata["race_imp"] = mydata["race_imp"].astype('category')
/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_13597/1563144068.py:5: SettingWithCopyWarning: 
A value is trying to be set on a 

participant_id       object
project                int8
specimen             object
collect_wk            int64
was_preterm            int8
was_early_preterm      int8
race_imp               int8
age_imp_cat            int8
dtype: object

We split the training and testing data set based on *project* J.

In [5]:
mydata_train = mydata[mydata['project']!=9] # Not project J
mydata_test  = mydata[mydata['project']==9] # project J

In [6]:
# mydata_train = mydata_train[["participant_id", "project", "specimen", "collect_wk", 
#                              "was_preterm", "was_early_preterm", "race_imp", "age_imp_cat" ]]

In [7]:
# mydata1["project"] = mydata1["project"].astype('category')
# mydata1["race_imp"] = mydata1["race_imp"].astype('category')
# mydata1["age_imp_cat"] = mydata1["age_imp_cat"].astype('category')

In [8]:
# mydata1["project"] = mydata1["project"].cat.codes
# mydata1["race_imp"] = mydata1["race_imp"].cat.codes
# mydata1["age_imp_cat"] = mydata1["age_imp_cat"].cat.codes
# mydata1['was_preterm'] = mydata1['was_preterm'].astype('int8')
# mydata1['was_early_preterm'] = mydata1['was_early_preterm'].astype('int8')
# mydata1.dtypes

We want to predict the class of preterm birth and early preterm birth for few consecutive specimemns collected. We will have to define the

- Input data (2 covariates)
    - The meta data (imputed age and race category) for each observation
- Output (1 covariate)
    - The class label of preterm birth/early preterm birth
    
via data pre-processing and move the data from numpy arrays to Pytorch data structure - **Torch Tensors**

## Preterm birth

Since the simple RNN only allow one covariates to enter the model, I choose *age_imp_cat* as a starting point.

### Input data from categorical age

We need to do some data filtering.

In [9]:
# only keep the rows with collect_wk < 37 for Preterm task
mydata1 = mydata_train.loc[mydata_train['collect_wk']<37,]

# delete the duplicated samples in the same collection week
mydata2 = mydata1.drop_duplicates(subset=['participant_id', 'collect_wk'], keep='first')
mydata2.shape

(2417, 8)

Note: Too much samples are deleted, we need to take care of this situation. For metadata table, its OK since the information of duplicates are exactly same. But it will be specimen specific for taxonomy, phylotypes and distance data sets. **We need to discuss it**. We need to either form up some summary statistic for multiple collected sample in the same week or other methods (???).

Since we will give a patient the label of preterm birth as the delievey week < 37, we have 36 as one of the dimension for the tensor.

In [10]:
mydata2_wide_age = mydata2.pivot_table(index=['participant_id'], columns='collect_wk', values='age_imp_cat')
# sort by collect_wk
mydata2_wide_age = mydata2_wide_age.sort_index(axis=1)
mydata2_wide_age = mydata2_wide_age.fillna(0)
mydata2_wide_age.shape
mydata2_wide_age.head

<bound method NDFrame.head of collect_wk       1    2    3    4    5    6    7    8    9    10  ...   27  \
participant_id                                                    ...        
A00001          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
A00003          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
A00004          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  2.0   
A00005          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
A00006          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
...             ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
I00534          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
I00535          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
I00536          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
I00537          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
I00538          0.0  0.0  0.0  0.0

In [11]:
(seq_length, num_feature) = mydata2_wide_age.shape
print(seq_length)
print(num_feature)

1174
36


### Output data from Preterm birth label

In [12]:
mydata2_wide_Y = mydata2.pivot_table(index=['participant_id'], columns='collect_wk', values='was_preterm')
# sort by collect_wk
mydata2_wide_Y = mydata2_wide_Y.sort_index(axis=1)
mydata2_wide_Y = mydata2_wide_Y.apply(lambda row: row.fillna(row.mean()), axis=1)
mydata2_wide_Y.head(100)

collect_wk,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
participant_id,,,,,,,,,,,,,,,,,,,,,
A00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C00026,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
C00027,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
C00028,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [13]:
# Creating lists that will hold our input and target sequences
input_seq = torch.Tensor([mydata2_wide_age.to_numpy()])
target_seq = torch.Tensor([mydata2_wide_Y.to_numpy()])

/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_13597/1327531968.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  input_seq = torch.Tensor([mydata2_wide_age.to_numpy()])


In [14]:
input_seq.shape

torch.Size([1, 1174, 36])

In [15]:
target_seq.shape

torch.Size([1, 1174, 36])

In [16]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [17]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, nonlinearity='relu')   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [18]:
# Instantiate the model with hyperparameters
model = Model(input_size=num_feature, output_size=num_feature, hidden_dim=12, n_layers=3)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 100
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [19]:
# Training Run
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output = output.to(device)
    target_seq = target_seq.to(device)
    loss = criterion(output, target_seq.view(-1, num_feature))
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/100............. Loss: 44.2118
Epoch: 20/100............. Loss: 44.1784
Epoch: 30/100............. Loss: 44.1764
Epoch: 40/100............. Loss: 44.1753
Epoch: 50/100............. Loss: 44.1747
Epoch: 60/100............. Loss: 44.1745
Epoch: 70/100............. Loss: 44.1744
Epoch: 80/100............. Loss: 44.1744
Epoch: 90/100............. Loss: 44.1744
Epoch: 100/100............. Loss: 44.1744


### Apply the simple RNN on testing set.

In [20]:
testset = mydata_test[["participant_id", "project", "specimen", "collect_wk", 
                       "was_preterm", "was_early_preterm", "race_imp", "age_imp_cat" ]]
print(testset.dtypes)
print(testset.shape)

participant_id       object
project                int8
specimen             object
collect_wk            int64
was_preterm            int8
was_early_preterm      int8
race_imp               int8
age_imp_cat            int8
dtype: object
(83, 8)


In [21]:
# only keep the rows with collect_wk < 37 for Preterm task
testset1 = testset.loc[testset['collect_wk']<37,]

# delete the duplicated samples in the same collection week
testset2 = testset1.drop_duplicates(subset=['participant_id', 'collect_wk'], keep='first')
testset2.shape

(83, 8)

In [22]:
testset2_wide_age = testset2.pivot_table(index=['participant_id'], columns='collect_wk', values='age_imp_cat')
# sort by collect_wk
testset2_wide_age = testset2_wide_age.sort_index(axis=1)
testset2_wide_age = testset2_wide_age.fillna(0)
testset2_wide_age.shape
testset2_wide_age.head

<bound method NDFrame.head of collect_wk       12   13   14   15   16   17   18   19
participant_id                                        
J00001          0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0
J00004          0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0
J00007          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
J00008          0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0
J00010          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
...             ...  ...  ...  ...  ...  ...  ...  ...
J00111          0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0
J00112          0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0
J00113          0.0  0.0  0.0  0.0  3.0  0.0  0.0  0.0
J00115          0.0  0.0  0.0  0.0  0.0  0.0  3.0  0.0
J00116          0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0

[83 rows x 8 columns]>

Fill-in zeros for empty collection weeks.

In [23]:
(seq_length_test, num_feature_test) = testset2_wide_age.shape

In [24]:
testset3 = pd.DataFrame(np.nan, index=range(seq_length_test), columns=range(num_feature))
testset3 = testset3.rename(columns={x:y for x,y in zip(testset3.columns,range(0,num_feature))})
testset3 = testset3.fillna(0)
print(testset3.shape)
test_index = testset2_wide_age._get_numeric_data().columns.values.tolist()
testset3.iloc[:,test_index] = testset2_wide_age

# check: first observation in testing set has value 3.0 at column 19 (index 18)
testset3.iloc[0]

(83, 36)


0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    3.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
Name: 0, dtype: float64

In [25]:
ii = 0
mytest_input = testset3.iloc[ii]
mytest_input = torch.Tensor([[mytest_input.to_numpy()]])
mytest_input.shape

torch.Size([1, 1, 36])

In [26]:
model.eval() # eval mode
out, hidden = model(mytest_input)
prob = nn.functional.softmax(out[-1], dim=0).data
# Taking the class with the highest probability score from the output
prob_max = torch.max(prob)
print(prob_max)
pred_label = 1*(prob_max > 0.5)
pred_label

tensor(0.0284)


tensor(0)

In [27]:
model.eval() # eval mode

predicted_labels = []
for ii in range(seq_length_test):
    mytest_input = testset3.iloc[ii]
    mytest_input = torch.Tensor([[mytest_input.to_numpy()]])
    out, hidden = model(mytest_input)
    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    prob_max = torch.max(prob).item()
    print(prob_max)
    pred_label = 1*(prob_max > 0.5)
    predicted_labels.append(pred_label)

0.028367556631565094
0.02836424857378006
0.028406862169504166
0.028367556631565094
0.028406862169504166
0.028377937152981758
0.028367556631565094
0.028425009921193123
0.028425851836800575
0.028354018926620483
0.028406862169504166
0.028427284210920334
0.028406862169504166
0.028377937152981758
0.028377937152981758
0.028367556631565094
0.028406862169504166
0.028354018926620483
0.028406862169504166
0.028377937152981758
0.028406862169504166
0.028425851836800575
0.028425009921193123
0.028425009921193123
0.028425009921193123
0.028425009921193123
0.028354018926620483
0.028406862169504166
0.028377937152981758
0.028377937152981758
0.028377937152981758
0.028354018926620483
0.02838703617453575
0.028367556631565094
0.028377937152981758
0.028354018926620483
0.028377937152981758
0.028386743739247322
0.028354018926620483
0.028377937152981758
0.028364913538098335
0.02836424857378006
0.028377937152981758
0.028377937152981758
0.028381386771798134
0.028377937152981758
0.02836424857378006
0.028404938057065

In [28]:
predicted_labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

Note: since we only involved *age_imp_cat* in this baby RNN model, all patients in project J are poredicted to not have preterm with a very small p-value.

In [29]:
testset2_wide_Y = testset2.pivot_table(index=['participant_id'], columns='collect_wk', values='was_preterm')
# sort by collect_wk
testset2_wide_Y = testset2_wide_Y.sort_index(axis=1)
actual_labels = testset2_wide_Y.mean(axis=1)
print(actual_labels.iloc[1:80])

participant_id
J00004    0.0
J00007    0.0
J00008    0.0
J00010    0.0
J00011    0.0
         ... 
J00107    0.0
J00108    0.0
J00109    0.0
J00111    0.0
J00112    0.0
Length: 79, dtype: float64


In [30]:
# create vectors for actual labels and predicted labels...
my_accuracy = accuracy_score(actual_labels, predicted_labels, normalize=False) / float(actual_labels.size)
my_accuracy

0.8192771084337349